# 02 - Experimentation - Local

This notebook covers the following steps:

1. Preparing the data using `NVTabular`.
2. Train, and evaluate the `TensorFlow` model.
3. Export a `TensorFlow` model.

## Setup

In [1]:
%env PROTOCOL_BUFFERS_PYTHON_IMPLEMENTATION=python
%env TF_MEMORY_ALLOCATION=0.7

env: PROTOCOL_BUFFERS_PYTHON_IMPLEMENTATION=python
env: TF_MEMORY_ALLOCATION=0.7


In [2]:
import os
import time
import logging
from datetime import datetime

import cudf
import nvtabular as nvt

import tensorflow as tf
import tensorflow.keras as keras

from src.common import features, utils
from src.data_preprocessing import etl
from src.model_training import trainer

logging.getLogger().setLevel(logging.INFO)
tf.get_logger().setLevel('INFO')

from google.protobuf.internal import api_implementation
print("protobuf implementation type:", api_implementation.Type())
print("TensorFlow:", tf.__version__)

protobuf implementation type: python
TensorFlow: 2.4.1


In [3]:
PROJECT = 'merlin-on-gcp'
REGION = 'us-central1'
BUCKET = 'merlin-on-gcp'
VERTEX_SERVICE_ACCOUNT = f'vertex-sa-mlops@{PROJECT}.iam.gserviceaccount.com'

MOVIES_CSV_DATASET_LOCATION = f"gs://{BUCKET}/movielens25m/dataset/movies.csv"
RATINGS_CSV_DATASET_LOCATION = f"gs://{BUCKET}/movielens25m/dataset/ratings.csv"

MODEL_DISPLAY_NAME = f'movielens25m-recommender'

LOCAL_WORKSPACE = '_workspace'
WORKSPACE = f"gs://{BUCKET}/movielens25m"
EXPERIMENT_ARTIFACTS_DIR = os.path.join(WORKSPACE, 'experiments')

TENSORBOARD_DISPLAY_NAME = f'tb-{PROJECT}'
EXPERIMENT_NAME = f'{MODEL_DISPLAY_NAME}-experiment'

## Initialize Experiment

In [4]:
REMOVE_EXPERIMENT_ARTIFACTS = False
if tf.io.gfile.exists(EXPERIMENT_ARTIFACTS_DIR) and REMOVE_EXPERIMENT_ARTIFACTS:
    print("Removing previous experiment artifacts...")
    tf.io.gfile.rmtree(EXPERIMENT_ARTIFACTS_DIR)

if not tf.io.gfile.exists(EXPERIMENT_ARTIFACTS_DIR):
    print("Creating new experiment artifacts directory...")
    tf.io.gfile.mkdir(EXPERIMENT_ARTIFACTS_DIR)

print("Preparing local workspace...")
if tf.io.gfile.exists(LOCAL_WORKSPACE):
    tf.io.gfile.rmtree(LOCAL_WORKSPACE)
tf.io.gfile.mkdir(LOCAL_WORKSPACE)
    
print("Workspace is ready.")

run_id = f"run-local-{datetime.now().strftime('%Y%m%d%H%M%S')}"
EXPERIMENT_RUN_DIR = os.path.join(EXPERIMENT_ARTIFACTS_DIR, EXPERIMENT_NAME, run_id)
print("Experiment run directory:", EXPERIMENT_RUN_DIR)

Preparing local workspace...
Workspace is ready.
Experiment run directory: gs://merlin-on-gcp/movielens25m/experiments/movielens25m-recommender-experiment/run-local-20210623161616


## 1. Preparing the data using NVTabular

In [5]:
ETL_OUTPUT_DIR = os.path.join(EXPERIMENT_RUN_DIR, 'etl_output')

In [6]:
etl.run_etl(
    PROJECT, 
    REGION,  
    MOVIES_CSV_DATASET_LOCATION, 
    RATINGS_CSV_DATASET_LOCATION, 
    ETL_OUTPUT_DIR)

INFO:root:Loading dataframes...
INFO:root:Dataframe loaded.
INFO:root:Movies data: (62423, 2).
INFO:root:Ratings data: (25000095, 3).
INFO:root:Splitting dataset to train and test splits...
INFO:root:Train split size: 20000076
INFO:root:Test split size: 5000019
INFO:root:Loading NVTabular datasets...
INFO:root:NVTabular datasets loaded.
INFO:root:Creating transformation workflow...
INFO:root:Fitting workflow to train data split...
INFO:root:Transformation workflow is fitted.
INFO:root:Transforming train dataset...
INFO:root:Train data is transformed.
INFO:root:Transforming test dataset...
INFO:root:Test dataset is transformed.
INFO:root:Saving transformation workflow...
INFO:root:Transformation workflow is saved.
INFO:root:Uploading trandorm workflow to Cloud Storage...
INFO:root:Transformation uploaded to Cloud Storage.


In [7]:
!gsutil ls {ETL_OUTPUT_DIR}

gs://merlin-on-gcp/movielens25m/experiments/movielens25m-recommender-experiment/run-local-20210623161616/etl_output/transform_workflow/
gs://merlin-on-gcp/movielens25m/experiments/movielens25m-recommender-experiment/run-local-20210623161616/etl_output/transformed_data/


## 2. Train a TensorFlow model

In [8]:
EXPORT_DIR = os.path.join(EXPERIMENT_RUN_DIR, 'model')

In [9]:
LOCAL_DATA_DIR = os.path.join(LOCAL_WORKSPACE, 'data')
LOCAL_TRAIN_DATA_DIR = os.path.join(LOCAL_DATA_DIR, 'train')
LOCAL_TEST_DATA_DIR = os.path.join(LOCAL_DATA_DIR, 'test')
LOCAL_MODEL_DIR = os.path.join(LOCAL_WORKSPACE, 'exported_model')

tf.io.gfile.mkdir(LOCAL_DATA_DIR)
tf.io.gfile.mkdir(LOCAL_TRAIN_DATA_DIR)
tf.io.gfile.mkdir(LOCAL_TEST_DATA_DIR)
tf.io.gfile.mkdir(LOCAL_MODEL_DIR)

### Prepare experiment parameters

In [10]:
hyperparams = {
    'learning_rate': 0.001,
    'batch_size': 1024 * 32,
    'hidden_units': [128, 128],
    'num_epochs': 1
}

### Download the data locally

In [11]:
utils.copy_files(os.path.join(ETL_OUTPUT_DIR, 'transformed_data', 'train', '*.parquet'), LOCAL_TRAIN_DATA_DIR)
utils.copy_files(os.path.join(ETL_OUTPUT_DIR, 'transformed_data', 'test', '*.parquet'), LOCAL_TEST_DATA_DIR)
utils.download_directory(os.path.join(ETL_OUTPUT_DIR, 'transform_workflow'), LOCAL_WORKSPACE)

### Train the model

In [12]:
nvt_workflow = nvt.Workflow.load(os.path.join(LOCAL_WORKSPACE, 'transform_workflow'))

In [13]:
recommendation_model = trainer.train(
    train_data_file_pattern=os.path.join(LOCAL_TRAIN_DATA_DIR, '*.parquet'),
    nvt_workflow=nvt_workflow,
    hyperparams=hyperparams
)

INFO:root:Hyperparameter:
INFO:root:{'learning_rate': 0.001, 'batch_size': 32768, 'hidden_units': [128, 128], 'num_epochs': 1}
INFO:root:
INFO:root:Preparing train dataset loader...
INFO:root:Embedding shapes: {'userId': (162542, 512), 'movieId': (56586, 512), 'genres': (21, 16)}
INFO:root:Compiling the model...
INFO:root:Model fitting started...


611/611 [==============================] - 22s 33ms/step - loss: 0.5418 - mae: 0.3659


INFO:root:Model fitting finished.


### Evaluate the model

In [14]:
eval_loss, eval_mse = trainer.evaluate(
    recommendation_model,
    eval_data_file_pattern=os.path.join(LOCAL_TEST_DATA_DIR, '*.parquet'),
    hyperparams=hyperparams
)

INFO:root:Preparing evaluation dataset loader...
INFO:root:Evaluating the model...


153/153 [==============================] - 2s 10ms/step - loss: 0.4888 - mae: 0.3226


INFO:root:Evaluation loss: 0.4887738823890686 - Evaluation MAE 0.3225647211074829


### Export the model

In [16]:
trainer.export(
    recommendation_model=recommendation_model,
    nvt_workflow=nvt_workflow,
    model_name=MODEL_DISPLAY_NAME,
    export_dir=LOCAL_MODEL_DIR
)

INFO:tensorflow:Assets written to: _workspace/exported_model/movielens25m-recommender_tf/1/model.savedmodel/assets


INFO:tensorflow:Assets written to: _workspace/exported_model/movielens25m-recommender_tf/1/model.savedmodel/assets


In [17]:
utils.upload_directory(LOCAL_MODEL_DIR, EXPORT_DIR)

In [18]:
!gsutil ls {EXPORT_DIR}

gs://merlin-on-gcp/movielens25m/experiments/movielens25m-recommender-experiment/run-local-20210623161616/model/movielens25m-recommender/
gs://merlin-on-gcp/movielens25m/experiments/movielens25m-recommender-experiment/run-local-20210623161616/model/movielens25m-recommender_nvt/
gs://merlin-on-gcp/movielens25m/experiments/movielens25m-recommender-experiment/run-local-20210623161616/model/movielens25m-recommender_tf/
